In [77]:
import torch
import numpy as np
import json
import os
import cv2
import random
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import tqdm

BLUE=(255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)
PINK = (255, 0, 255)
BLACK = (0, 0, 0)
ORANGE = (0, 127, 255)
CUSTOM = (255,170,170)
COLOR_CLASS = {0: BLUE, 1:GREEN, 2:RED, 3:YELLOW, 4:PINK, 5:BLACK, 6:ORANGE, 7:CUSTOM}

# def bb_intersection_over_union(boxA, boxB):
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])
#     interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

#     boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
#     boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

#     iou = interArea / float(boxAArea + boxBArea - interArea)
#     return iou

def cocoToAbsoluteBox(cocoBox):
    #xywh -> xyxy
    return [cocoBox[0], cocoBox[1], cocoBox[0]+cocoBox[2], cocoBox[1]+cocoBox[3]]


# matplotlib 
def plot(key, list_bbox_):
    fig = plt.figure(figsize=(15,10))

    plt.hist(list_bbox_[key], color = 'blue', edgecolor = 'black',
            bins = int(len(set(list_bbox_[key]))))

    # Add labels
    plt.title('Histogram of {}'.format(key))
    plt.xlabel(key)
    plt.ylabel('count')
#     plt.savefig("d/{}_distribution.png".format(key))


def bb_intersection_over_union(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    bb1 = {'x1':bb1[0], 'x2':bb1[2], 'y1':bb1[1], 'y2':bb1[3]}
    bb2 = {'x1':bb2[0], 'x2':bb2[2], 'y1':bb2[1], 'y2':bb2[3]}
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [78]:
x = [100,100,110,110]
y = [90, 88,99,99]
bb_intersection_over_union(y,x)

0.0

# load gt

In [79]:
df_gt = pd.read_csv('//home/hana/sonnh/kaggle-vin/dataset/images_only/train_only_box.csv')
df_gt.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,fold
0,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,1780.0,361.0,2047.0,612.0,4
1,e7e8948818352b4d800dfac9a8999300,Pleural thickening,11,R9,599.0,303.0,885.0,366.0,4
2,53e2a10eb9969b0e336a51d11dda17f9,Pleural thickening,11,R8,1838.0,370.0,2416.0,897.0,1
3,5bf3368744630f459a499ccdccc9cdf1,Cardiomegaly,3,R10,1066.0,2026.0,2324.0,2586.0,0
4,0f186e3eba8d9ebd51feed957204ddbf,Cardiomegaly,3,R8,1192.0,1695.0,2321.0,2009.0,1


In [80]:
data_gt = {}
for i in range(len(df_gt)):
    image_id = df_gt.iloc[i]['image_id']
    if image_id not in data_gt:
        data_gt[image_id] = []
    
    xmin = df_gt.iloc[i]['x_min']
    xmax = df_gt.iloc[i]['x_max']
    ymin = df_gt.iloc[i]['y_min']
    ymax = df_gt.iloc[i]['y_max']
    class_id = df_gt.iloc[i]['class_id']
    data_gt[image_id].append([xmin, ymin, xmax, ymax, class_id])


# data gt to data_layer 3

In [97]:
def data_gt_to_layer3(df_gt, fold):
    df_gt_fold = df_gt[df_gt['fold'] == (fold-1)]
    
    print('positive {}'.format(len(df_gt_fold)))
    
    df_gt_to_layer3 = {}
    for i in range(len(df_gt_fold)):

        image_id = df_gt_fold.iloc[i]['image_id']
        if image_id not in df_gt_to_layer3:
            df_gt_to_layer3[image_id] = {'bbox':[], 'label':[], 'conf':[], 'yolo_label' :[]}

        xmin, ymin, xmax, ymax = int(df_gt_fold.iloc[i]['x_min']), int(df_gt_fold.iloc[i]['y_min']), int(df_gt_fold.iloc[i]['x_max']), int(df_gt_fold.iloc[i]['y_max'])

        df_gt_to_layer3[image_id]['bbox'].append([xmin, ymin, xmax, ymax])
        df_gt_to_layer3[image_id]['label'].append([int(df_gt_fold.iloc[i]['class_id'])])

        yolo_class = [0]*14
        yolo_class[int(df_gt_fold.iloc[i]['class_id'])] = 1
        df_gt_to_layer3[image_id]['yolo_label'].append(yolo_class)
        
        

        conf = [0] * 14
        conf[int(df_gt_fold.iloc[i]['class_id'])] = 1
        
        df_gt_to_layer3[image_id]['conf'].append(conf)
    return df_gt_to_layer3


# data predict

In [98]:
def create_data_predict(data):
    data_predict = {}
    for anno in data:
    #     {'image_id': '852ec6a0bcb9608ffee9b3fda0867f91', 'category_id': 0, 'bbox': [1632.083, 878.747, 348.558, 422.076], 'score': 0.74072, conf:[]}
        image_id = anno['image_id']
        if image_id not in data_predict:
            data_predict[image_id] = {}
            data_predict[image_id]['bbox'] = []   
            data_predict[image_id]['label'] = []   
            data_predict[image_id]['conf'] = []   
            data_predict[image_id]['yolo_label'] = []   

        xmin = int(anno['bbox'][0])
        ymin = int(anno['bbox'][1])
        w = int(anno['bbox'][2])
        h = int(anno['bbox'][3])

        if w < 10 or h < 10:
            continue

        xmax = xmin + w
        ymax = ymin + h
        data_predict[image_id]['bbox'].append([xmin, ymin, xmax, ymax])
        data_predict[image_id]['conf'].append(anno['conf'])
        
        yolo_class = [0]*14
        yolo_class[anno['category_id']] = 1
        data_predict[image_id]['yolo_label'].append(yolo_class)
        
    return data_predict

# create data layer3

In [99]:
from tqdm import tqdm

In [100]:
# data_layer_3
def thong_ke_layer_3(data_gt, data_predict, no_box = False):
    c = 0
    list_c_ = []
    data_label = {}
#     for image_id in tqdm(data_predict):
    for image_id in data_predict:
        for i, bbox in enumerate(data_predict[image_id]['bbox']):
            data_predict[image_id]['label'].append([])
            check = False
            c_ = 0
            if not no_box:
                for bbox_gt in data_gt[image_id]:
                    if bb_intersection_over_union(bbox, bbox_gt)> 0.4:
                        check = True
                        data_predict[image_id]['label'][i].append(bbox_gt[-1])

                        c_ += 1

            list_c_.append(c_)
            if  check: c += 1
        
    print('negative {} positive {}'.format(list_c_.count(0), c))
    return data_predict

# remove box

In [75]:
json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/runs_eval_layer3/fold{}.json'.format(fold)
data = json.load(open(json_path, 'r'))
data_overlay = {}
for anno in data:
    print(anno)
#     image_id = 
#     data_overlay
    break

{'image_id': '91e721ea0143333c5efe5dfb8ca98746', 'category_id': 0, 'bbox': [1537.177, 288.766, 270.241, 305.474], 'score': 0.38208, 'conf': [0.382080078125, 0.00045943260192871094, 0.0007810592651367188, 0.0014448165893554688, 0.0003781318664550781, 0.00043845176696777344, 0.0005965232849121094, 0.0007138252258300781, 0.0007305145263671875, 0.0011892318725585938, 0.0003108978271484375, 0.0004968643188476562, 0.00030732154846191406, 0.0008053779602050781]}


In [ ]:
def remove_box(data):
    for anno in data:
        

# run

In [106]:
### for fold in [1,2,3,4, 5]:
fold = 1
print('yolo predict')
json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/runs_eval_layer3/fold{}.json'.format(fold)
data = json.load(open(json_path, 'r'))
data_predict = create_data_predict(data)
data_predict = thong_ke_layer_3(data_gt, data_predict, no_box =False)
# # negative 61531 positive 20640
# # negative 79545 positive 23146
# # negative 50836 positive 19810
# # negative 69187 positive 21270
# # negative 89664 positive 19791

print('gt')
df_gt = pd.read_csv('//home/hana/sonnh/kaggle-vin/dataset/images_only/train_only_box.csv')
data_gt_fold = data_gt_to_layer3(df_gt, fold)
for image_id in data_gt_fold:
    data_predict[image_id]['bbox'] += data_gt_fold[image_id]['bbox']
    data_predict[image_id]['conf'] += data_gt_fold[image_id]['conf']
    data_predict[image_id]['label'] += data_gt_fold[image_id]['label']
    data_predict[image_id]['yolo_label'] += data_gt_fold[image_id]['yolo_label']

print('yolo predict no box')
json_path = '/home/hana/sonnh/kaggle-vin/final/1920_pretrain_train_all/runs_eval_layer3_nobox/fold{}_0.005.json'.format(fold)
# negative 16223 positive 0
# negative 18981 positive 0
# negative 12356 positive 0
# negative 19965 positive 0
# negative 22919 positive 0
data = json.load(open(json_path, 'r'))
data_predict_no_box = create_data_predict(data)
data_predict_no_box = thong_ke_layer_3(data_gt, data_predict_no_box, no_box =True)

data_predict.update(data_predict_no_box)


print()

yolo predict
negative 31800 positive 15298
gt
positive 7220
yolo predict no box
negative 15092 positive 0



# cut image

In [107]:
def cut_data(image_id):
    image_dir = '/home/hana/sonnh/kaggle-vin/dataset/images_only/train_jpg/'
    l3_dir = '/home/hana/sonnh/kaggle-vin/dataset/layer_3_1920_kfold/data_cut/fold1'
    
    img_ori = cv2.imread('{}/{}.jpg'.format(image_dir, image_id))
    height, width = img_ori.shape[:2]
    
    gt = []
    
    for i in range(len(data_predict[image_id]['bbox'])):
        
        anno_gt = {'image_id': '{}_{}'.format(image_id, i)}
        
        anno = data_predict[image_id]['bbox'][i]
        label = data_predict[image_id]['label'][i]
        
        xmin, ymin, xmax, ymax = anno
        small_image = img_ori[ymin:ymax, xmin:xmax]
        small_image = cv2.resize(small_image, (256, 256))
        
        cv2.imwrite('{}/{}_{}.jpg'.format(l3_dir, image_id, i), small_image)

        list_label= [0]*15
        if len(label) == 0:
            list_label[-1] = 1
            
        for singe_label in label:
            list_label[singe_label] += 1
            
        anno_gt['label'] = list_label
        
        xmin /= width
        xmax /= width
        ymin /= height
        ymax /= height
        anno_gt['conf'] = data_predict[image_id]['conf'][i] + [xmin, ymin, xmax, ymax]
        gt.append(anno_gt)
    return gt

In [108]:
import os
from multiprocessing import Pool

pool = Pool()
data = pool.map(cut_data, list(data_predict.keys()))

In [109]:
data_final = []
for i in range(len(data)):
    data_final += data[i]
len(data_final)

#1 48384
#2 48070
#3 48469
#4 48577
#5 49121

69410

In [48]:
json_path = '/home/hana/sonnh/kaggle-vin/dataset/layer_3/data_cut_from_pretrain_all_5fold/anno/fold_{}.json'.format(fold)
with open(json_path, 'w') as out_file:
    json.dump(data_final, out_file)